# Анализ авторства текстов
Этот блокнот анализирует два текстовых файла (`author_1.txt` и `author_2.txt`) и оценивает вероятность того, что они написаны одним автором.

## Установка зависимостей
Установите библиотеки (выполните в терминале или через !):

In [ ]:
# !pip install nltk fuzzywuzzy python-Levenshtein matplotlib spacy scikit-learn pandas
# !python -m spacy download ru_core_news_sm

## Импорт библиотек

In [16]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams
from collections import Counter
import re
from fuzzywuzzy import fuzz
import spacy
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, cross_val_score

# Загрузка данных NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Загрузка модели spacy для русского языка
nlp = spacy.load('ru_core_news_sm')
nlp = spacy.load('ru_core_news_sm')
nlp.max_length = 2000000  # Достаточно для 1,446,486 символов
# Установка отображения графиков
%matplotlib inline

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alexewd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexewd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Функции для анализа

In [ ]:
# Чтение файла
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Очистка текста
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^а-яё\s]', '', text)  # Оставляем только русские буквы и пробелы
    return text

# Частотность слов
def word_frequency(text):
    words = word_tokenize(clean_text(text))
    stop_words = set(stopwords.words('russian'))
    words = [word for word in words if word and word not in stop_words]  # Убираем пустые строки
    if not words:  # Отладка
        print("Ошибка: список слов пустой после токенизации")
        return Counter()
    return Counter(words)

# Биграммы
def ngram_frequency(text, n=2):
    words = word_tokenize(clean_text(text))
    stop_words = set(stopwords.words('russian'))
    words = [word for word in words if word and word not in stop_words]
    n_grams = ngrams(words, n)
    return Counter(n_grams)

# Анализ частей речи (весь текст)
def pos_analysis(text):
    doc = nlp(text)
    pos_counts = Counter([token.pos_ for token in doc])
    return pos_counts

# Длина предложений
def sentence_length_analysis(text):
    sentences = sent_tokenize(text)
    lengths = [len(word_tokenize(sent)) for sent in sentences]
    return sum(lengths) / len(lengths) if lengths else 0, lengths

# Нечёткое сравнение
def fuzzy_comparison(text1, text2, sample_size=1000):
    text1_chunk = clean_text(text1)[:sample_size]
    text2_chunk = clean_text(text2)[:sample_size]
    return fuzz.ratio(text1_chunk, text2_chunk)

# Вероятность единого авторства
def authorship_probability(fuzzy_score, accuracy, common_words, total_words1, total_words2, avg_len1, avg_len2):
    fuzzy_weight = fuzzy_score / 100  # 30% веса
    classifier_weight = (1 - accuracy) * 0.5 + 0.5  # 20% веса
    common_words_ratio = common_words / min(total_words1, total_words2)  # 40% веса
    len_diff = abs(avg_len1 - avg_len2) / max(avg_len1, avg_len2)
    len_weight = 1 - min(len_diff, 1)  # 10% веса
    probability = (fuzzy_weight * 0.3 + classifier_weight * 0.2 + common_words_ratio * 0.4 + len_weight * 0.1)
    return min(max(probability, 0), 1)

## Полный анализ и вывод вероятности

In [11]:
# Пути к файлам
file1 = "f:\\chatepc\\chatalx\\work\\data\\obriv.txt"
file2 = "f:\\chatepc\\chatalx\\work\\data\\oblomov.txt"

# Чтение текстов
text1 = read_file(file1)
text2 = read_file(file2)
print(f"Первые 100 символов {file1}: {text1[:100]}")
print(f"Первые 100 символов {file2}: {text2[:100]}")

# 1. Частотность слов
freq1 = word_frequency(text1)
freq2 = word_frequency(text2)
top_words1 = pd.DataFrame(freq1.most_common(10), columns=['Слово', 'Частота'])
top_words2 = pd.DataFrame(freq2.most_common(10), columns=['Слово', 'Частота'])
print(f"Топ-10 слов в {file1}:")
display(top_words1)
print(f"Топ-10 слов в {file2}:")
display(top_words2)
common_words = len(set(freq1.keys()) & set(freq2.keys()))
print(f"Количество общих слов: {common_words}")

# 2. Биграммы
bigrams1 = ngram_frequency(text1, n=2)
bigrams2 = ngram_frequency(text2, n=2)
top_bigrams1 = pd.DataFrame([( ' '.join(bg), freq) for bg, freq in bigrams1.most_common(5)], columns=['Биграмма', 'Частота'])
top_bigrams2 = pd.DataFrame([( ' '.join(bg), freq) for bg, freq in bigrams2.most_common(5)], columns=['Биграмма', 'Частота'])
print(f"Топ-5 биграмм в {file1}:")
display(top_bigrams1)
print(f"Топ-5 биграмм в {file2}:")
display(top_bigrams2)

# 3. Анализ частей речи (весь текст)
print("Анализ частей речи может занять несколько минут...")
pos1 = pos_analysis(text1)  # Обрабатываем весь текст
pos2 = pos_analysis(text2)
total_pos1 = sum(pos1.values())
total_pos2 = sum(pos2.values())
pos_df1 = pd.DataFrame(
    [(pos, freq, freq / total_pos1 * 100) for pos, freq in pos1.items()],
    columns=['Часть речи', 'Частота', 'Процент']
).sort_values(by='Частота', ascending=False)
pos_df2 = pd.DataFrame(
    [(pos, freq, freq / total_pos2 * 100) for pos, freq in pos2.items()],
    columns=['Часть речи', 'Частота', 'Процент']
).sort_values(by='Частота', ascending=False)
print(f"Части речи в {file1} (весь текст):")
display(pos_df1)
print(f"Части речи в {file2} (весь текст):")
display(pos_df2)

# 4. Длина предложений
avg_len1, lengths1 = sentence_length_analysis(text1)
avg_len2, lengths2 = sentence_length_analysis(text2)
print(f"Средняя длина предложений в {file1}: {avg_len1:.2f} слов")
print(f"Средняя длина предложений в {file2}: {avg_len2:.2f} слов")

# 5. Нечёткое сходство
fuzzy_score = fuzzy_comparison(text1, text2)

# 6. Классификатор Naive Bayes
vectorizer = TfidfVectorizer(max_features=2000, ngram_range=(1, 2))
X = vectorizer.fit_transform([text1, text2])
sentences1 = sent_tokenize(text1)
sentences2 = sent_tokenize(text2)
X_sentences = vectorizer.transform(sentences1 + sentences2)
y_sentences = [0] * len(sentences1) + [1] * len(sentences2)
X_train, X_test, y_train, y_test = train_test_split(X_sentences, y_sentences, test_size=0.3, random_state=42, stratify=y_sentences)
clf = MultinomialNB()
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
scores = cross_val_score(clf, X_sentences, y_sentences, cv=5)

# 7. Итоговая вероятность
total_words1 = len(set(word_tokenize(clean_text(text1))))
total_words2 = len(set(word_tokenize(clean_text(text2))))
prob = authorship_probability(fuzzy_score, accuracy, common_words, total_words1, total_words2, avg_len1, avg_len2)

# 8. Таблица итоговых метрик
results_df = pd.DataFrame({
    'Метрика': ['Нечёткое сходство (%)', 'Точность Naive Bayes (тест)', 'Средняя точность (кросс-валидация)', 'Вероятность единого авторства (%)'],
    'Значение': [fuzzy_score, f"{accuracy:.2f}", f"{scores.mean():.2f} (±{scores.std():.2f})", f"{prob:.2%}"]
})
print("\nИтоговые метрики анализа:")
display(results_df)

# Визуализация
plt.figure(figsize=(10, 6))
plt.hist(lengths1, bins=20, alpha=0.5, label=file1)
plt.hist(lengths2, bins=20, alpha=0.5, label=file2)
plt.legend(loc='upper right')
plt.title('Распределение длины предложений')
plt.xlabel('Количество слов')
plt.ylabel('Частота')
plt.show()

pos1_labels, pos1_values = zip(*pos1.items())
pos2_labels, pos2_values = zip(*pos2.items())
plt.figure(figsize=(12, 6))
plt.bar(pos1_labels, pos1_values, alpha=0.5, label=file1)
plt.bar(pos2_labels, pos2_values, alpha=0.5, label=file2)
plt.legend(loc='upper right')
plt.title('Распределение частей речи')
plt.xlabel('Часть речи')
plt.ylabel('Частота')
plt.xticks(rotation=45)
plt.show()

Части речи в f:\chatepc\chatalx\work\data\obriv.txt:


,Часть речи,Частота
0,NOUN,42
8,PUNCT,30
2,ADJ,21
4,VERB,16
5,ADP,16
6,ADV,12
3,NUM,11
7,PROPN,9
10,CCONJ,9
1,SPACE,6


Части речи в f:\chatepc\chatalx\work\data\oblomov.txt:


,Часть речи,Частота
4,NOUN,49
6,PUNCT,27
2,ADP,24
5,ADJ,15
8,VERB,14
0,PROPN,10
11,CCONJ,10
1,SPACE,7
12,ADV,7
3,NUM,6
